参考: [Python で並列処理(初めての人向け)](http://iatlex.com/python/parallel_first)  

In [3]:
import multiprocessing
from multiprocessing import Pool

multiprocessing.cpu_count()

4

### 一番簡単な形

In [5]:
def square(x):                        # 引数が1つの関数から扱ってみる
    print(x * x)

p = Pool(4)                             # 4つのコアを使って
p.map(square, range(10))   # 0 ~ 9までの2乗計算を並列で実行

0
1
4
16
25
9
36
49
81
64


[None, None, None, None, None, None, None, None, None, None]

このように，並列計算は終わる順番が決まっているわけではないことに注意する  
なお，Poolに引数を指定しなければcpu_countの個数使う

### 引数が複数の関数の並列化

In [7]:
def times(a, b):
    return a * b

def wrapper_times(args):             # 複数の値を渡すにはラッパーが必要
    return times(*args)                    # mapは1つの引数の関数しか基本的に扱えない

args = [[i, 3] for i in range(10)]   # 0 ~ 9の値を三倍
p = Pool(processes = 2)                # 2並行で処理
p.map(wrapper_times, args)        # 実行結果はリストで帰ってくる

[0, 3, 6, 9, 12, 15, 18, 21, 24, 27]

[Python高速化 【multiprocessing】【並列処理】](https://qiita.com/taka-kawa/items/d1fc1bc0acb3a6ca3031)曰く  
- ある程度重い処理でなければ並列かの恩恵を受けられない

以下参考: [Python並列処理(multiprocessingとJoblib)](https://qiita.com/yukiB/items/203a6248c2d466b80d38)

In [1]:
from joblib import Parallel, delayed

def process(i):
    return [{'id': j, 'sum': sum(range(i * j))} for j in range(100)]

result = Parallel(n_jobs=-1, verbose=5)( # -1でコア数全て, verboseは0~10の頻度の度合い
    [delayed(process)(n) for n in range(1000)] # delayedは非同期処理
)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 860 tasks      | elapsed:   16.7s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:   22.2s finished


次のように，複数の値を直接受け取れたり，リスト以外の値も受け取ることができるのがjoblibの利点である．

In [ ]:
strs = ['a', 'b', 'c']
result = Parallel(n_jobs=job)([delayed(process)(i,s) for i,s in enumerate(strs * 1000)])

しかし，測ってみるとmultiprocessingの方が若干早い模様